In [1]:
%pip install langchain_community
%pip install bs4
%pip install pypdf
%pip install faiss-cpu
%pip install chromadb
%pip install langchain_ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Data Ingestion

## Text Loader

In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("song.txt")

In [2]:
# Read the data

text_documents = loader.load()
text_documents

[Document(metadata={'source': 'song.txt'}, page_content='â€¦ I had a dream\nI got everything I wanted\nNot what you\'d think\nAnd if I\'m being honest\nâ€¦ It might\'ve been a nightmare\nTo anyone who might care\nThought I could fly (fly)\nSo I stepped off the Golden, mm\nNobody cried (cried, cried, cried, cried)\nNobody even noticed\nI saw them standing right there\nKinda thought they might care (might care, might care)\nâ€¦ I had a dream\nI got everything I wanted\nBut when I wake up, I see\nYou with me\nâ€¦ And you say, "As long as I\'m here\nNo one can hurt you\nDon\'t wanna lie here\nBut you can learn to\nIf I could change\nThe way that you see yourself\nYou wouldn\'t wonder why you hear\nThey don\'t deserve you"\nâ€¦ I tried to scream\nBut my head was underwater\nThey called me weak\nLike I\'m not just somebody\'s daughter\nâ€¦ Coulda been a nightmare\nBut it felt like they were right there\nAnd it feels like yesterday was a year ago\nBut I don\'t wanna let anybody know\n\'Cause 

In [4]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="C:/Users/Lenovo/Desktop/LANGCHAIN/.env", override=True)

True

## Web Based Loader

In [1]:
# Data Ingestion - Web Based Loader

from langchain_community.document_loaders import WebBaseLoader
import bs4

got_wiki = WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/Game_of_Thrones",),
                          bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="mw-content-text")))

got_wiki_docs = got_wiki.load()
got_wiki_docs


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_content='American fantasy drama TV series (2011–2019)\n"GoT" redirects here. For other uses, see GOT.\nThis article is about the TV series. For other uses, see Game of Thrones (disambiguation).\n\n\nGame of ThronesTitle card for the first seven seasonsGenre\nAction\nAdventure\nFantasy[1]\nSerial drama[2]\nTragedy[3][4]\nCreated by\nDavid Benioff\nD. B. Weiss\nBased onA Song of Ice and Fireby George R. R. MartinShowrunners\nDavid Benioff\nD. B. Weiss\nStarringsee List of Game of Thrones charactersTheme music composerRamin DjawadiOpening theme"Main Title"ComposerRamin DjawadiCountry of originUnited StatesOriginal languageEnglishNo. of seasons8No. of episodes73 (list of episodes)ProductionExecutive producers\nDavid Benioff\nD. B. Weiss\nGeorge R. R. Martin\nCarolyn Strauss\nFrank Doelger\nBernadette Caulfield\nBryan Cogman\nMiguel Sapochnik\nDavid Nutter\nProducers\nMark Huffam\nJoanna Burn\nChris Newman\

## PDF Loader

In [6]:
from langchain_community.document_loaders import PyPDFLoader

book_loader = PyPDFLoader("attention.pdf")

book_docs = book_loader.load()
book_docs[:5]

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

# Data Transformation

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
got_docs_split = text_splitter.split_documents(got_wiki_docs)

got_docs_split[:5]

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_content='American fantasy drama TV series (2011–2019)\n"GoT" redirects here. For other uses, see GOT.\nThis article is about the TV series. For other uses, see Game of Thrones (disambiguation).'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_content='Game of ThronesTitle card for the first seven seasonsGenre\nAction\nAdventure\nFantasy[1]\nSerial drama[2]\nTragedy[3][4]\nCreated by\nDavid Benioff\nD. B. Weiss\nBased onA Song of Ice and Fireby George R. R. MartinShowrunners\nDavid Benioff\nD. B. Weiss'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_content='David Benioff\nD. B. Weiss\nStarringsee List of Game of Thrones charactersTheme music composerRamin DjawadiOpening theme"Main Title"ComposerRamin DjawadiCountry of originUnited StatesOriginal languageEnglishNo. of seasons8No. of episodes73 (list of episodes)ProductionExec

# Vector Embedding & Vector Store

In [4]:
from langchain_ollama import OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(got_docs_split[:5], embedding=OllamaEmbeddings(model="llama3"))

In [5]:
query = "Who is Ramin Djawadi?"
results = db.similarity_search(query)
results

[Document(id='93d92080-3fad-490f-8d8a-f352a3dd73e8', metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_content='Game of Thrones is an American fantasy drama television series created by David Benioff and D. B. Weiss for HBO. It is an adaptation of A Song of Ice and Fire, a series of high fantasy novels by George R. R. Martin, the first of which is A Game of Thrones. The show premiered on HBO in the United States on April 17, 2011, and concluded on May 19, 2019, with 73 episodes broadcast over eight seasons.'),
 Document(id='34b18f90-25da-4059-a901-91cb5d7746c0', metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_content='American fantasy drama TV series (2011–2019)\n"GoT" redirects here. For other uses, see GOT.\nThis article is about the TV series. For other uses, see Game of Thrones (disambiguation).'),
 Document(id='5d2ea051-016e-4f48-b0e3-413ebf6e382e', metadata={'source': 'https://en.wikipedia.org/wiki/Game_of_Thrones'}, page_conten